<img src="unicamp.png" width="150" height="150">

## MO444/MC886 - Aprendizado de Máquina e Reconhecimento de Padrões

Esse trabalho foi feito pelos seguintes membros:

- Lucas Zanco Ladeira - 188951
- Rafael Scherer - 204990

O código original deste projeto está disponível em [repository inside Github](https://github.com/lucaslzl/p1_clustering). 

## Modelos de Agrupamento e Redução de Dimensionalidade

## I - Introdução

Neste trabalho foi necessário implementar dois modelos de agrupamento e utilizar o algoritmo <b>Principal Component Analysis (PCA)</b> da biblioteca scikit-learn para a tarefa de redução de dimensionalidade. Os modelos implementados compreendem o <b>KMeans</b> e o <b>DBScan</b>. De forma resumida, no primeiro caso são selecionados centróides de clusters e analisados os clusters formados por esses centróides. O algoritmo é iterado um determinado número de vezes e clusters são encontrados na qual a distância entre os membros dos clusters e os centróides seja mínima dentro das iterações. No segundo caso, é calculada a distância entre todos os registros para encontrar quais formam clusters considerando uma distância máxima e uma vizinhança mínima. Os registros são classificados como <i>outlier</i>, borda, e central. Sendo assim, é um algoritmo interessante para identificar <i>outliers</i> dentro de conjuntos de dados.

## II - Implementação

Nesta seção será descrito o código fonte dos algoritmos implementados. Para tal, será sub-dividia em: a) KMeans, b) DBScan, c) PCA. Mesmo sendo que foi utilizada uma biblioteca para implementar o PCA, o código fonte será disponibilizado neste relatório.

### II - a) KMeans

Esse algoritmo possui dois métodos principais, seguindo o padrão utilizado pela biblioteca scikit-learn, esses compreendem <i>fit</i> e <i>predict</i>. O primeiro tem o intuito de treinar o modelo, ou seja, agrupar os dados em K clusters. <br>
Inicialmente, os centroides dos K clusters são inicializados utilizando um de dois métodos disponíveis:
- 'forgy': são escolhidos K centroides aleatórios a partir do conjunto de dados.
- 'kmeans++', onde são escolhidos K centroides com base em uma lógica probabilística.

Após a inicialização dos centroides, é iniciado o agrupamento dos dados em clusters. Para cada ponto pertencente ao conjunto de dados, é calculada a distância entre o ponto e cada centroide definido, em seguida, o ponto é atribuido ao cluster do centroide cuja distância calculada foi mínima.

Para fins de análise do comportamento do algoritmo, foi criado um dicionário 'history', que armazena algumas informações relevantes para cada iteração do algoritmo, sendo elas: centroides, pontos pertencentes a cada clusters e inércia.

Por fim, o algoritmo conta com duas possíveis condições de parada:
- Caso o número de iterações realizadas alcance o limite definido por <i>max_iter</i>
- Caso os centroides dos K clusters convirjam, ou seja, parem de se mover em alguma iteração.

In [ ]:
def fit(self, data):
    # data initialization
    self._data_init(data)

    for i in range(self.max_iter):
        # assign the previous centroids
        previous_centroids = dict(self.centroids)

        # initialize the clusters dictionary
        for label in range(self.k):
            self.clusters[label] = []

        # assign each data point to a cluster
        for data_point in data:
            distances = self._centroid_distance(data_point)
            # the data point is assigned to the nearest cluster
            label = np.argmin(distances)
            self.clusters[label].append(data_point)

        # the position of each centroid is moved to the mean of the points in the cluster
        for label, cluster in self.clusters.items():
            # compute the mean only if the cluster is not empty
            if cluster:
                self.centroids[label] = np.mean(cluster, axis=0)

        # compute inertia
        self._compute_inertia()

        # save current centroids and clusters to history
        self.history[i] = {'centroids': dict(self.centroids), 'clusters': dict(self.clusters), 'inertia': self.inertia}

        # check if the algorithm has converged
        converged = True
        for label, centroid in self.centroids.items():
            previous_centroid = previous_centroids[label]
            centroid_distance = distance.euclidean(centroid, previous_centroid)
            # if the distance between the current and previous centroid is greater than zero
            # then the algorithm hasn't converged yet
            if centroid_distance > 0:
                converged = False

        # if the algorithm converged then the learning process is finished
        if converged:
            break

Alguns métodos auxiliares são chamados durante o treinamento do modelo, são eles:
- data_init: realiza a inicialização dos centroides com base no método escolhido.

In [ ]:
def _data_init(self, data):
    if self.init == 'kmeans++':
        self._kmeans_plus_plus_init(data)
    else:
        self._forgy_init(data)

def _forgy_init(self, data):
    # chooses k random points from the data as initial centroids
    np.random.seed(self.random_seed)
    random_centroids = np.random.choice(len(data), self.k, replace=False)

    for i in range(self.k):
        self.centroids[i] = data[random_centroids[i]]

def _kmeans_plus_plus_init(self, data):
    np.random.seed(self.random_seed)
    # the first centroid is chosen at random
    centroids = [data[np.random.choice(len(data))]]

    for i in range(self.k - 1):
        dx_array = []
        for data_point in data:
            centroid_distances = []
            # compute the distance from the data point to each centroid
            for centroid in centroids:
                centroid_distance = distance.euclidean(data_point, centroid)
                centroid_distances.append(centroid_distance)

            # dx denotes the square of the shortest distance from the data point to a centroid
            dx = np.min(centroid_distances) ** 2
            dx_array.append(dx)

        # compute the probabilities
        square_sum = np.sum(dx_array)
        probabilities = np.divide(dx_array, square_sum)
        # the point with the highest probability is the new centroid
        highest_probability = np.max(probabilities)
        new_centroid = data[np.where(probabilities == highest_probability)][0]
        centroids.append(new_centroid)

    for i in range(self.k):
        self.centroids[i] = centroids[i]

- centroid_distance: dado um ponto pertencente ao conjunto de dados,
retorna uma lista contendo as distâncias do ponto a cada um dos centroides definidos.

In [ ]:
def _centroid_distance(self, data_point):
        # compute the euclidean distance from the data point to each centroid
        distances = []
        for _, centroid in self.centroids.items():
            centroid_distance = distance.euclidean(data_point, centroid)
            distances.append(centroid_distance)

        return distances

- compute_inertia: calcula a inércia, medida definida como a soma do quadrado das distâncias de cada ponto ao seu centroide mais próximo.

In [ ]:
def _compute_inertia(self):
        # compute the inertia (sum of squared distances of samples to their closest centroid)
        inertia = 0
        for label, centroid in self.centroids.items():
            for point in self.clusters[label]:
                error = distance.euclidean(centroid, point)
                inertia += error ** 2
        self.inertia = inertia

O segundo método principal utilizado pelo algoritmo é o predict, que tem como objetivo realizar a predição de novos dados,
 ou seja, atribuir cada registro pertencente aos dados a algum dos clusters encontrados durante o treinamento.<br>
Para isso, é utilizado o mesmo método para cálculo de distâncias entre pontos e centroides utilizado durante o treinamento.
Portanto, cada registro é atribuido ao cluster para qual sua distância seja mínima.

In [ ]:
def predict(self, data):
        predict_data = []
        for data_point in data:
            # the data point is assigned to the nearest cluster
            distances = self._centroid_distance(data_point)
            label = np.argmin(distances)
            predict_data.append(label)

### II - b) DBScan

Esse algoritmo possui dois métodos principais, seguindo o padrão utilizado pela biblioteca scikit-learn, esses compreendem <i>fit</i> e <i>predict</i>. O primeiro tem o intuito de treinar o modelo, ou seja, identificar o comportamento dos registros. Os registros são iterados e classificados de acordo com <i>outlier</i>, borda ou central. Além disso, caso seja um registro central é atribuído a um novo cluster. As variáveis nc (node classification) e ci (cluster id) armazenam essas informações. A seguir, é descrito este método com comentários na língua inglesa para facilitar a extensão a partir da disponibilização do código implementado.

In [ ]:
def fit(self, x):

    # Initialize with 0's
    nc = [0] * len(x)
    ci = [0] * len(x)

    cluster_id = 1

    # Iterate through all records
    for i in tqdm(range(len(x))):

        # If already classified, skip
        if nc[i] != 0:
            continue

        # Get neighbors
        neighbors = self._get_neighbors(x, i)

        # Verify if it is an outlier
        if len(neighbors) < self.min_neighbors:
            nc[i] = -1
            continue

        # Core record
        nc[i] = 2
        ci[i] = cluster_id

        # Iterate through each neighbor
        indx = 0
        while True:

            # If list of neighbors ended
            if indx == len(neighbors):
                break

            j = neighbors[indx]

            # Verify if neighbor is classified as outlier
            if nc[j] == -1:
                nc[j] = 1
                ci[j] = cluster_id

            # Verify if neighbor was already classified
            if nc[j] != 0:
                indx += 1
                continue

            post_neighbors = self._get_neighbors(x, j)

            # Verify if neighbor is core point
            if len(post_neighbors) >= self.min_neighbors:

                nc[j] = 2
                ci[j] = cluster_id

            else:
                # Classify as border point
                nc[j] = 1
                ci[j] = cluster_id

            # Continue exploring neighbourhood
            neighbors.extend(post_neighbors)
            neighbors = list(set(neighbors))

            indx += 1

        cluster_id += 1

    return (nc, ci)

É possível observar algumas chamadas para o método <i>_get_neighbors<i>. Este método tem o intuito de buscar todos os vizinhos de um determinado registro. Um método chamado <i>_verify_neighbor</i> faz o cálculo da distância euclidiana entre dois registros e retorna <i>True</i> se for vizinho, e <i>False</i> se não for vizinho.

In [ ]:
def _verify_neighbor(self, point_a, point_b):

    # Calculate euclidean distance
    calc_dist = distance.euclidean(point_a, point_b)

    # Append distance to verify description
    self.summed_dist.append(calc_dist)

    # Verify if it is a neighbor
    if calc_dist <= self.distance:
        return True, self.distance

    return False, self.distance


def _get_neighbors(self, x, i):
    
    # Neighbor list
    neighbors = []

    for j in range(len(x)):

        if i == j:
            continue

        # Verify if it is a neighbor
        verif, _ = self._verify_neighbor(x[i], x[j])
        
        if verif:
            # Append to the list of neighbors
            neighbors.append(j)

    return neighbors

Agora será descrito o método <i>predict</i> que faz a predição dos clusters para novos registros considerando os registros centrais já identificados. O método faz a identificação de qual é o registro central mais próximo.

In [ ]:
def predict(self, x, res, y):
    
    (nc, ci) = res
    
    # Initialize with 0's
    ci_pred = [0] * len(y)

    for i in tqdm(range(len(y))):

        # Get neighbors
        neighbors = self._get_neighbors_predict(x, i, y)

        if len(neighbors) > 0:

            # Get closest neighbors
            neighbors = self._get_by_closest(neighbors)

            for j in range(len(neighbors)):
                
                # Get closest neighbors
                indx_j = int(neighbors[j][0])

                # Verify if core point
                if nc[indx_j] == 2:
                    ci_pred[i] = ci[indx_j]
                    break

    return ci_pred

Este método utiliza um método distinto para obter os vizinhos, pois é necessário considerar cada vizinho, como também, as distâncias para os vizinhos. O método tem nome <i>_get_neighbors_predict</i>. Um outro método, chamado <i>_get_by_closest</i>, é utilizado para ordenar todos os vizinhos de acordo com a distância calculada.

In [ ]:
def _get_neighbors_predict(self, x, i, y):

    # Neighbor list
    neighbors = []

    for j in range(len(x)):

        # Verify if it is a neighbor
        verif, dist = self._verify_neighbor(y[i], x[j])

        if verif:
            neighbors.append((j, dist))

    return neighbors


def _get_by_closest(self, neighbors):

    neighbors = np.array(neighbors)
    return neighbors[neighbors[:, 1].argsort()]

### II - c) Principal Componente Analysis

O algoritmo do PCA foi implementado utilizando a biblioteca [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). De acordo com a [wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis) o "<i>Principal component analysis (PCA) is the process of computing the principal components and using them to perform a change of basis on the data, sometimes using only the first few principal components and ignoring the rest</i>". Sendo assim, caso seja necessário reduzir um conjunto de dados para 3 dimensões é necessário apenas obter os 3 principais componentes. Para facilitar a utilização, foi criada uma classe nova chamada <i>OurPCA</i> com o método <i>fit_transform</i>, o qual cria um objeto PCA e faz a transformações nos dados.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

class OurPCA:

    def fit_transform(self, data, n_components):

        pca = PCA(n_components=n_components, random_state=42)
        return pca.fit_transform(data)

Além das classes e métodos descritos, outros algoritmos foram implementados para gerenciar os dados, resultados, e experimentos. Estes compreendem:
- <i>main.py</i><br>
Une tudo o que foi implementado para executar os experimentos.<br>


- <i>inout.py</i><br>
Encapsula métodos de leitura de arquivos, persistência de resultados, transformações nos dados, e criação de gráficos.

## III - Metodologia de Avaliação

### III - a) Bases de dados

Neste trabalho são utilizadas duas bases de dados. A primeira foi disponibilizada pela professora Esther, e possui 2 <i>features</i> numéricas. Para descrever os registros da base de dados o método <i>describe</i> da biblioteca pandas é utilizada.

In [1]:
import pandas as pd

df = pd.read_csv('datasets/cluster.dat', sep=' ')

df.describe()

,x,y
count,573.000000,573.000000
mean,1849.808028,15.227836
std,900.129972,8.292268
min,335.000000,1.950000
25%,1155.000000,7.450000
50%,1655.000000,17.200000
75%,2350.000000,22.750000
max,3635.000000,29.150000


A segunda base de dados se refere a registros de históricos de cartões de crédito. O intuito da tarefa de agrupamento é identificar perfis de usuários para campanhas de marketing. Essa base de dados foi obtida do [Kaggle](https://www.kaggle.com/arjunbhasin2013/ccdata). Ela possui 18 <i>features</i> numéricas com alguns valores nulos e 8950 registros. Os valores nulos são preenchidos com 0's.

In [3]:
df = pd.read_csv('datasets/credit.csv')

df.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8.950000e+03,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8.950000e+03,8.637000e+03,8950.000000,8950.000000
mean,1620.986304,7.714003,1003.204834,592.437371,411.067645,3.270640e+03,4.452865,2.030237,4.117664,2.214069,3.248827,14.709832,4494.449450,5.404793e+03,1.809551e+03,3.809273,11.517318
std,4385.370311,73.859272,2136.634782,1659.887917,904.338115,1.311675e+05,52.604114,29.744300,54.021898,32.210553,6.824647,24.857649,3638.815725,1.702333e+05,3.856941e+04,47.047820,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000e+00,1.916300e-02,0.000000,6.000000
25%,128.281915,0.900000,39.635000,0.000000,0.000000,0.000000e+00,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,3.833158e+02,1.692297e+02,0.000000,12.000000
50%,874.387676,1.000000,361.280000,38.000000,89.000000,0.000000e+00,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,8.577677e+02,3.128075e+02,0.000000,12.000000
75%,2056.395445,1.000000,1110.130000,577.405000,468.637500,1.114220e+03,1.000000,0.333333,0.750000,0.250000,4.000000,17.000000,6500.000000,1.906756e+03,8.260137e+02,0.166667,12.000000
max,311357.000000,875.000000,49039.570000,40761.250000,22500.000000,1.024992e+07,875.000000,875.000000,875.000000,1125.000000,123.000000,358.000000,30000.000000,1.089444e+07,2.559345e+06,875.000000,12.000000


### III - b) Experimentos

## IV - Resultados
### Experimento 1: KMeans, primeiro dataset
<img src="plots/d0_elbow_kmeans.png">
<img src="plots/d0_silhouette_kmeans_2.png">
<img src="plots/d0_silhouette_kmeans_3.png">
<img src="plots/d0_silhouette_kmeans_4.png">
<img src="plots/d0_silhouette_kmeans_5.png">
<img src="plots/d0_silhouette_kmeans_6.png">
<img src="plots/d0_silhouette_kmeans_7.png">
<img src="plots/d0_silhouette_kmeans_8.png">
<img src="plots/d0_silhouette_kmeans_9.png">
<img src="plots/d0_silhouette_kmeans_10.png">
<img src="plots/d0_silhouette_kmeans_11.png">
A partir da análise do resultado da aplicação do método do cotovelo no primeiro dataset, verifica-se que 3 é o número de clusters
mais provável para melhor representar os dados. Tal hipótese é reforçada pela análise dos resultados do método da silhueta, que também
apresentou 3 como sendo o possível melhor número de clusters, visto que, dado o intervalo analisado,
o coeficiente da silhueta é maior em K = 3, assumindo o valor de 0.6963021999511755.

### Experimento 2: KMeans, primeiro dataset
#### Utilizando inicialização forgy
<img src="plots/d0_cluster_iteration_not_scaled_forgy_1.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_2.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_3.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_4.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_5.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_6.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_7.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_8.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_9.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_10.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_11.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_12.png">
<img src="plots/d0_cluster_iteration_not_scaled_forgy_13.png">
<img src="plots/d0_inter_cluster_distance_not_scaled_forgy.png">
<img src="plots/d0_intra_cluster_distance_not_scaled_kmeans.png">
<img src="plots/d0_clusters_predict_not_scaled_forgy.png">


#### Utilizando inicialização kmeans++
<img src="plots/d0_cluster_iteration_not_scaled_kmeans_1.png">
<img src="plots/d0_cluster_iteration_not_scaled_kmeans_2.png">
<img src="plots/d0_cluster_iteration_not_scaled_kmeans_3.png">
<img src="plots/d0_inter_cluster_distance_not_scaled_kmeans.png">
<img src="plots/d0_intra_cluster_distance_not_scaled_kmeans.png">
<img src="plots/d0_clusters_predict_not_scaled_kmeans.png">

Nota-se que, independente da inicialização, o algoritmo não foi capaz de separar os clusters adequadamente
durante os experimentos realizados com dados não normalizados, entretanto, percebe-se que, apesar do resultado incorreto,
o algoritmo foi convergiu mais rapidamente ao utilizar a inicialização kmeans++.
Ademais, nota-se que a distância entre clusters apresentou um comportamento indesejado com ambas inicializações,
no caso da inicialização forgy, a distância atingiu um máximo na iteração 6 porém diminuiu nas iterações seguintes, enquanto
que com a inicialização kmeans++, a distância atingiu um máximo na primeira iteração, porém também caiu nas iterações seguintes.

### Experimento 3: KMeans, primeiro dataset
#### Utilizando inicialização forgy
<img src="plots/d0_cluster_iteration_scaled_forgy_1.png">
<img src="plots/d0_cluster_iteration_scaled_forgy_2.png">
<img src="plots/d0_cluster_iteration_scaled_forgy_3.png">
<img src="plots/d0_cluster_iteration_scaled_forgy_4.png">
<img src="plots/d0_inter_cluster_distance_scaled_forgy.png">
<img src="plots/d0_intra_cluster_distance_scaled_forgy.png">
<img src="plots/d0_clusters_predict_scaled_forgy.png">

#### Utilizando inicialização kmeans++
<img src="plots/d0_cluster_iteration_scaled_kmeans_1.png">
<img src="plots/d0_cluster_iteration_scaled_kmeans_2.png">
<img src="plots/d0_inter_cluster_distance_scaled_kmeans.png">
<img src="plots/d0_intra_cluster_distance_scaled_kmeans.png">
<img src="plots/d0_clusters_predict_scaled_kmeans.png">

Diferentemente do experimento anterior, nota-se que, com os dados normalizados, o algoritmo conseguiu separar os clusters
de forma adequada. Novamente, percebe-se que o algoritmo convergiu mais rapidamente ao utilizar a inicialização kmeans++,
caso onde a convergência ocorreu em apenas 2 iterações, ou seja, a inicialização kmeans++ foi capaz de escolher os melhores
centros para o conjunto de dados.
Em relação as inter e intra-clusters, nota-se que, diferente do experimento anterior, a evolução ocorreu da forma esperada, ou seja,
a distância inter-cluster aumentou e a distância intra-cluster diminuiu ao longo das iterações

## V - Conclusões

## VI - Apêndice

<span style="color: red">Links, figuras, etc</span>

### Links

- Scikit-learn (https://scikit-learn.org/stable/)